In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 7
model = Network('model')

In [3]:
# Create network
delay = Delay(model,'Delay')
fork1 = Fork(model,'Fork1')
join1 = Join(model,'Join1',fork1)
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)

jobclass1 = ClosedClass(model, 'class1', 1, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 1, delay, 0)

In [4]:
# Service configurations
delay.set_service(jobclass1, Exp(0.25))
queue1.set_service(jobclass1, Exp(2.0))
queue2.set_service(jobclass1, Exp(2.0))

delay.set_service(jobclass2, Exp(0.25))
queue1.set_service(jobclass2, Exp(2.0))
queue2.set_service(jobclass2, Exp(2.0))

In [5]:
# Set routing matrix with class switching
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass1, jobclass1, delay, fork1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join1, 1.0)
P.set(jobclass1, jobclass1, queue2, join1, 1.0)
P.set(jobclass1, jobclass1, join1, delay, 1.0)

# Class 2 routing (note the class switching from jobclass2 to jobclass1)
P.set(jobclass2, jobclass2, delay, fork1, 1.0)
P.set(jobclass2, jobclass1, fork1, queue1, 1.0)
P.set(jobclass2, jobclass1, fork1, queue2, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [MVA(model), JMT(model, seed=23000)] # JMT has a bug on this one
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: MVA
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   class1 0.8270 0.8270 4.0000  4.0000 0.2068 0.2068
  Join1   class1 0.1153 0.0000 0.2788  0.2788 0.2068 0.2068
  Join1   class2 0.6667 0.0000 0.4167  0.4167 0.8000 0.8000
 Queue1   class1 0.1153 0.1034 0.5577  0.5577 0.2068 0.2068
 Queue1   class2 0.6667 0.4000 0.8333  0.8333 0.8000 0.8000
 Queue2   class1 0.1153 0.1034 0.5577  0.5577 0.2068 0.2068
 Queue2   class2 0.6667 0.4000 0.8333  0.8333 0.8000 0.8000
SOLVER 2: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   class1 0.8317 0.8317 3.9359  3.9359 0.2103 0.2103
  Delay   class2 0.8266 0.8266 3.9266  3.9266 0.2108 0.2108
  Join1   class1 0.2406 0.2202 0.2850  0.2850 0.2103 0.2103
  Join1   class2 0.0000 0.0000 0.0000  0.0000 0.2108 0.2108
 Queue1   class1 0.2355 0.2130 0.5454  0.5454 0.4141 0.4141
 Queue1   class2 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000
 Queue2   class1 0.2319 0.2082 0.5466  0.5466 0.4211 0.4211
 Queue2   class2 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000
